<h3>The Below 3 Cells Use SQLAlchemy and Pandas to create a table based on the above datafram</h3>

In [577]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table
from sqlalchemy import create_engine, inspect
from sqlalchemy import MetaData
from edgerdb import helper_functions as hlp
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process


meta = MetaData()

engine = create_engine('postgresql://analyst:@localhost:5432/edgar')

messages = Table('filings', meta, autoload=True, autoload_with=engine)

def sql_to_df(query, connection):
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [394]:
conn = engine.connect()
conn.close()


In [395]:
def create_csi_data_table(engine):
    '''
    Creates a table with the data from csidata.com
    '''
    conn = engine.connect()
    csi_stock_data = pd.read_csv('http://www.csidata.com/factsheets.php?type=stock&format=csv')
    csi_stock_data.columns = map(str.lower, csi_stock_data.columns)
    csi_stock_data = csi_stock_data.rename(columns={'name': 'company_name'})
    csi_stock_data['company_name'] = csi_stock_data['company_name'].str.upper()
    csi_stock_data['company_name'] = csi_stock_data['company_name'].str.strip()
    csi_stock_data = csi_stock_data.fillna(value="NONE")
    csi_stock_data.to_sql('csi_stock_data', engine)
    conn.close()
    

In [14]:
def create_table_with_cik_and_csi_data():
    '''
        Creates a table called cik_to_csi by joining filings and csi_stock_data tables on company_name
    '''
    join_ticker_to_cik = """select * into temp_cik_to_csi from (select fil.cik, fil.company_name,
                            csi.symbol, csi.exchange,
                            csi.isactive, csi.startdate, csi.enddate
                            from filings fil
                            join csi_stock_data csi on fil.company_name = csi.company_name) as foo;"""
    clean_ticker_to_cik_table = """select * into cik_to_csi
    from (select distinct cik, c.company_name, c.symbol, c.exchange, c.isactive, c.startdate, c.enddate  from temp_cik_to_csi c) as foo;"""
    hlp.clear_sessions('edgar', hlp.db())
    hlp.statement(join_ticker_to_cik, hlp.db(), output=False, commit=True)
    hlp.statement(clean_ticker_to_cik_table, hlp.db(), output=False, commit=True)
    hlp.statement("drop table temp_cik_to_csi;", hlp.db(), output=False, commit=True)
    

In [15]:
filings_data = pd.read_sql_query('select * from filings;', con=engine)
temp_csi_data = pd.read_sql_query('select * from csi_stock_data;', index_col='index',con=engine)
temp_csi_data = temp_csi_data.rename(columns={'name': 'company_name'})
filings_data = filings_data.rename(columns={'company': 'company_name'})
csi_data = pd.DataFrame(csi_stock_data, columns=['csinumber', 'symbol', 'company_name', 'exchange', 'isactive', 'startdate', 'enddate'])
csi_data = csi_data.dropna()
csi_data.head()

,csinumber,symbol,company_name,exchange,isactive,startdate,enddate
0,1001,DPS,DR PEPPER/7UP,NYSE,0,1993-01-27,1995-06-06
1,1002,PTB,PARAGON TRADE BRANDS INC,NYSE,0,1993-01-27,1999-07-07
2,1003,WLP,WELLPOINT HEALTH NETWORKS INC,NYSE,0,1993-01-28,2004-12-02
3,1004,ARN,AMERICAN RE CORP,NYSE,0,1993-01-29,1996-11-25
4,1005,BWS,BETA WELL SERVICES,AMEX,0,1993-02-01,1994-11-30


In [80]:
csi_data[csi_data['company_name'].str.contains('EQUITY')].head()

,csinumber,symbol,company_name,exchange,isactive,startdate,enddate
11,1012,ELS,EQUITY LIFESTYLE PROPERTIES INC,NYSE,1,1993-02-25,2016-10-28
89,1090,EQR,EQUITY RESIDENTIAL,NYSE,1,1993-08-12,2016-10-28
254,1255,TYW,TAIWAN EQUITY FUND,NYSE,0,1994-07-19,2000-05-05
273,1274,KEF,KOREA EQUITY FUND INC,NYSE,1,1994-09-19,2016-10-28
411,1412,EQU,EQUITY CP INTL,NYSE,0,1994-10-19,1999-01-19


In [19]:
#new_df = csi_data.join(filings_data, on='company_name', how='inner')


In [17]:
engine = create_engine('postgresql://analyst:@localhost:5432/edgar')

conn = engine.connect()

cik_to_csi_df = pd.read_sql_table('cik_to_csi', conn)
conn.close()

In [18]:
# Equivelent of sql where clause
cik_to_csi_df[cik_to_csi_df['cik']  == '1000045']

,cik,company_name,symbol,exchange,isactive,startdate,enddate
1,1000045,NICHOLAS FINANCIAL INC,NICK,NASDAQ,1,1997-12-29,2016-10-19
2,1000045,NICHOLAS FINANCIAL INC,NICU,VSE,0,1996-08-08,1998-03-11


In [19]:
# What you need to do is figure out which stocks are in the csi data but did not map to a company name from the sec
# Some of the names may have been entered different
# Natural language processing may help

In [20]:
def sql_to_df(query, connection):
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [21]:
cik_to_csi_symbols = sql_to_df("select symbol from cik_to_csi;", engine.connect())['symbol'].values.tolist()


In [22]:
# Create a table to 
csi_stock_data_not_found_in_filings = sql_to_df("""select distinct symbol, 
                                company_name, exchange, isactive, 
                                startdate, enddate 
                                from csi_stock_data where symbol not in {};""".format(tuple(cik_to_csi_symbols)), 
                           engine.connect())

In [23]:
# Creates a table of the unmatching companies
#csi_stock_data_not_found_in_filings.to_sql('unmatching_csi_data', engine)

In [24]:
csi_stock_data = sql_to_df("select * from cik_to_csi;", engine.connect())

In [357]:
csi_stock_data.head()

,cik,company_name,symbol,exchange,isactive,startdate,enddate
0,1000015,META GROUP INC,METG,NASDAQ,0,1995-12-01,2005-03-31
1,1000045,NICHOLAS FINANCIAL INC,NICK,NASDAQ,1,1997-12-29,2016-10-19
2,1000045,NICHOLAS FINANCIAL INC,NICU,VSE,0,1996-08-08,1998-03-11
3,1000048,TECHNOLOGY FLAVORS & FRAGRANCES INC,TFF,AMEX,0,1999-07-12,2005-06-28
4,1000048,TECHNOLOGY FLAVORS & FRAGRANCES INC,TFF,TSX,0,1996-08-13,2002-11-15


In [30]:
csi_stock_data_not_found_in_filings.head()

,symbol,company_name,exchange,isactive,startdate,enddate
0,&AGIN,AGI NORTH AMERICA,INDEX,0,2007-06-27,2013-03-21
1,&AGIZL,ARDOUR GLOBAL INDEX EXTRA LIQUID,INDEX,1,2007-06-27,2016-10-19
2,&CRSPSCT,CRSP US SMALL CAP TR INDEX,INDEX,1,2015-10-23,2016-10-19
3,&GSM,S&P GSTI SEMICONDUCTOR INDEX,INDEX,1,1996-07-08,2016-03-04
4,&GSO,S&P GSTI SOFTWARE INDEX,INDEX,1,1996-07-08,2016-10-19


In [27]:
filings_df = sql_to_df("select distinct company_name, cik from filings;", engine.connect())
filings_df['company_name'] = filings_df['company_name'].str.upper()
filings_df = filings_df[filings_df['company_name'] != ""]

In [ ]:
%%time

import re
from pickle import dump
from requests import get


def tickers_to_cik(list_of_tickers):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile('.*CIK=(\d{10}).*')

    cik_dict = {}
    for ticker in list_of_tickers:
        results = CIK_RE.findall(get(URL.format(ticker)).text)
        if len(results):
            cik_dict[str(ticker).upper()] = str(results[0]).lstrip('0')
        else:
            no_matches.append(str(ticker).upper())
    return cik_dict

def new_nasdaq_tickers_and_names(engine):
    """
        This function queries the cik_to_csi table and returns a dataframe of tickers and company names for companies
        not found in our data.
    """
    nasdaq_companies = pd.read_csv("data/nasdaqcompanylist.csv")
    amex_companies = pd.read_csv("data/amexcompanylist.csv")
    nyse_companies = pd.read_csv("data/nysecompanylist.csv")
    nasdaq_name_and_ticker = pd.DataFrame(nasdaq_companies, columns=['Symbol', 'Name'])
    nasdaq_name_and_ticker['exchange'] = 'NASDAQ'
    amex_name_and_ticker = pd.DataFrame(amex_companies, columns=['Symbol', 'Name'])
    amex_name_and_ticker['exchange'] = 'AMEX'
    nyse_name_and_ticker = pd.DataFrame(nyse_companies, columns=['Symbol', 'Name'])
    nyse_name_and_ticker['exchange'] ='NYSE'
    companies_and_symbols = pd.concat([nasdaq_name_and_ticker, amex_name_and_ticker, nyse_name_and_ticker])
    companies_and_tickers = companies_and_symbols.rename(columns={'Symbol': 'symbol', "Name": "name"})
    companies_and_tickers['isactive'] = 1
    list_of_tickers_from_nasdaq = companies_and_tickers['symbol'].values.tolist()
    list_of_tickers_in_cik_to_csi_table = pd.read_sql_query('select distinct symbol from cik_to_csi;',con=engine)['symbol'].values.tolist()
    not_found_in_db = []
    for x in list_of_tickers_from_nasdaq:
        if x not in list_of_tickers_in_cik_to_csi_table and x not in not_found_in_db:
            not_found_in_db.append(x)
    return companies_and_tickers[companies_and_tickers['symbol'].isin(not_found_in_db)]


def return_matches(first_li, second_li):
    return set(first_li) & set(second_li)


def remove_ending(company_name, ending):
    """
        Provide a company name and the phrase you are looking to end at.
    """
    a, b, c = company_name.partition(ending)
    return a + b

def remove_unimportant_characters(company_name):
    company_name = re.sub(r'\([^)]*\)', '', company_name)
    word_list = ['GROUP', 'INC', 'CORP', 'FUND', 'INTERNATIONAL']
    for word in word_list:
        company_name = re.sub(r'\b{}\b'.format(word), '', company_name)
    p = re.compile("&|-|\)|\(")
    company_name = p.sub('', company_name)
    return company_name
    
def normalized_company_namelist(df):
    company_names = {}
    common_phrases = []#['GROUP', 'INC', 'CORP', 'FUND', 'INTERNATIONAL']
    for x in df['company_name'].index:
        name = df['company_name'][x]
        for word in common_phrases:
            if word in name:
                no_inc = remove_ending(name, word)
                company_names[remove_unimportant_characters(no_inc)] = name
            else:
                pass
        company_names[remove_unimportant_characters(name)] = name
        
    return company_names


def insert(df, entries):
    """
        Takes a set of values and inserts them into a dataframe sequentially where the index is
        sequential and continuous.
    """
    try:
        df.loc[max(df.index) + 1] = entries
    except ValueError:
        df.loc[0] = entries
        
        
def shorten_to_match(dic, matches):
    new_dict = dic.copy()
    for key in dic.keys():
        if key not in matches:
            new_dict.pop(key)
    return new_dict
    

def matches_to_frame():
    normalized_csi = normalized_company_namelist(csi_stock_data_not_found_in_filings)
    normalized_filings = normalized_company_namelist(filings_df)
    new_matches = return_matches(list(normalized_csi.keys()), list(normalized_filings.keys()))
    matched_csi = shorten_to_match(normalized_csi, new_matches)
    matched_filings = shorten_to_match(normalized_filings, new_matches)
    
    new_matches_df = pd.DataFrame(columns=['filings_name', 'csi_name', 'match'])
    
    for val in new_matches:
        insert(new_matches_df, [matched_filings[val], matched_csi[val], val])
        
            
    return new_matches_df
    
def create_matches_dataframe(): 
    """
        This function creates a dataframe that finds new ticker matches from the csi data using fuzzystring algorithms.
        This method isn't very accurate but it's a start.
    """
    new_matches_df = matches_to_frame()
    merged_filings = pd.merge(left=new_matches_df, right=filings_df, right_on='company_name', left_on='filings_name')
    merged_filings = pd.DataFrame(merged_filings, columns=['filings_name', 'csi_name', 'match', 'cik'])
    merged_ticker = pd.merge(left= merged_filings, right=csi_stock_data_not_found_in_filings, right_on='company_name', left_on='csi_name')
    merged_ticker =  pd.DataFrame(merged_ticker, columns=['filings_name', 'csi_name', 'match','percent_match', 'cik', 'symbol'])

    def update_percent_match(merged_ticker):
        for x in range(len(merged_ticker)):
            merged_ticker.loc[x, ('percent_match')] = fuzz.ratio(merged_ticker.loc[x]['filings_name'], merged_ticker.loc[x]['csi_name'])
        
        return merged_ticker 

    merging_new_cik_csi = update_percent_match(merged_ticker)
    return merging_new_cik_csi



<h2>The Goal In The Next Section Is To Add as Many Tickers To CIK Id's as Possible</h2>

<ul>
    <li>Find as many matching tickers as possible</li>
    <li>Take the matching tickers, cik, and company names from all datasources and create a new table.</li>
    <li>Write a code that will automate this process</li>
</ul>

In [739]:
# This function will find mathces based on fuzzy string algorithms
matches_df = create_matches_dataframe()
len(matches_df)

2203

In [682]:
# This function finds tickers listed on the nasdaq website it should be 100% 
# correct and the company names match very closely to the sec filings
tickers_from_nasdaq_site = new_nasdaq_tickers_and_names(engine)
tickers_from_nasdaq_site = tickers_from_nasdaq_site.rename(columns={"name": "company_name"})
tickers_from_nasdaq_site['company_name'] = tickers_from_nasdaq_site['company_name'].str.upper()
tickers_from_nasdaq_site.head()


,symbol,company_name,exchange,isactive
0,PIH,"1347 PROPERTY INSURANCE HOLDINGS, INC.",NASDAQ,1
1,FLWS,"1-800 FLOWERS.COM, INC.",NASDAQ,1
2,FCCY,1ST CONSTITUTION BANCORP (NJ),NASDAQ,1
4,VNET,"21VIANET GROUP, INC.",NASDAQ,1
5,TWOU,"2U, INC.",NASDAQ,1


In [683]:
print("tickers_from_nasdaq_site: {}".format(len(tickers_from_nasdaq_site)))
print("Filings_df: {}".format(len(filings_df)))


tickers_from_nasdaq_site: 3909
Filings_df: 640632


In [684]:
filings_df.head()


,company_name,cik
1,!J INC,1438823
2,"#1 A LIFESAFER HOLDINGS, INC.",1509607
3,#1 ARIZONA DISCOUNT PROPERTIES LLC,1457512
4,#1 PAINTBALL CORP,1433777
5,$ LLC,1427189


In [685]:
# There was a loss of 2000 securities during this operation
matches_from_exchange_list= pd.merge(left=tickers_from_nasdaq_site, right=filings_df, right_on='company_name', left_on='company_name')

In [691]:
print(len(matches_from_exchange_list))


1933


In [754]:
#tickers_to_cik(['TWLO', 'BAC', 'AAPL'])
def make_list_of_tickers():
    ticker_list = []
    ticker_list = tickers_from_nasdaq_site['symbol'].values.tolist()
    csi_data_symbols = csi_data['symbol'].values.tolist()
    for tick in csi_data_symbols:
    #for tick in cik_to_csi_df['symbol'].values.tolist():
        if tick not in ticker_list:
            ticker_list.append(tick)
    return ticker_list

In [755]:
ticker_list = make_list_of_tickers()
ticker_cik_dict = tickers_to_cik(ticker_list[1:10])

In [756]:
ticker_list = make_list_of_tickers()
ticker_cik_dict = tickers_to_cik(ticker_list[1:10])
print(len(ticker_list))
ticker_cik_dict


99343


{'abeo': '318306',
 'avhi': '39677',
 'cafd': '1635581',
 'fccy': '1141807',
 'flws': '1084869',
 'jobs': '1295484',
 'shlm': '87565',
 'twou': '1459417',
 'vnet': '1508475'}

In [580]:
conn = engine.connect()
csi_to_cik_tick_matches = sql_to_df("select cik, company_name, symbol, exchange, isactive from cik_to_csi;",conn)
conn.close()

In [692]:
csi_to_cik_tick_matches.head()

,cik,company_name,symbol,exchange,isactive
0,1000015,META GROUP INC,METG,NASDAQ,0
1,1000045,NICHOLAS FINANCIAL INC,NICK,NASDAQ,1
2,1000045,NICHOLAS FINANCIAL INC,NICU,VSE,0
3,1000048,TECHNOLOGY FLAVORS & FRAGRANCES INC,TFF,AMEX,0
4,1000048,TECHNOLOGY FLAVORS & FRAGRANCES INC,TFF,TSX,0


In [694]:
cik_to_ticker = pd.concat([matches_from_exchange_list, csi_to_cik_tick_matches])
print(len(cik_to_ticker))
cik_to_ticker.head()

13909


,cik,company_name,exchange,isactive,symbol
0,1591890,"1347 PROPERTY INSURANCE HOLDINGS, INC.",NASDAQ,1,PIH
1,1508475,"21VIANET GROUP, INC.",NASDAQ,1,VNET
2,1459417,"2U, INC.",NASDAQ,1,TWOU
3,1295484,"51JOB, INC.",NASDAQ,1,JOBS
4,1295484,"51JOB, INC.",NASDAQ,1,JOBS


In [753]:
csi_data_symbols = csi_data['symbol'].values.tolist()

In [334]:
# The goal here is to write an algorithm that will find keys that mostly match in both list
def hash_list(li):
    dic = {}
    for x in li:
        dic[x] = x.split(" ")
    return dic

#return_matches(normalized_csi.keys(), normalized_filings.keys())
def check_for_matching_phrases(df, list1, list2):
    '''
        This function takes in two dictionaries and checks which keys contain similar values. 
        The match percentage is found by dividing the length of the minimum set of values by 
        the matches. Always put the largest dictionary first.
    '''
    #df = pd.DataFrame(columns=['filings_name', 'no_match_name', 'percent_match'])
    for value1 in list1:
        for value2 in list2:
            match_ratio = fuzz.ratio(value1, value2)
            partial_match_ratio = fuzz.partial_ratio(value1, value2)
            insert(df, [value1, value2, match_ratio, partial_match_ratio])
    return df



In [385]:
%%time
# This ran for 8 hours and was only half way finished with 343324 rows in the dataframe
#df = pd.DataFrame(columns=['filings_name', 'no_match_name', 'percent_match'])
#matching_phrases = check_for_matching_phrases(df, hashed_comp_fil, hashed_no_match)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


In [330]:
len(df)

2665

In [384]:
#df[df['percent_match'] > .9]

In [549]:
import pyximport; pyximport.install()
pyximport.install(pyimport = True)

(None, None)

In [360]:
from CheckMatches import check_matches as cm

In [386]:
%%time
# This should be importing everything in as Cython now

df = pd.DataFrame(columns=['filings_name', 'no_match_name', 'percent_match'])
matching_phrases = check_for_matching_phrases(df, hashed_comp_fil, hashed_no_match)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


In [553]:


def check_for_matching_phrases(df, list1, list2):
    '''
        This function takes in two dictionaries and checks which keys contain similar values. 
        The match percentage is found by dividing the length of the minimum set of values by 
        the matches. Always put the largest dictionary first.
    '''
    #df = pd.DataFrame(columns=['filings_name', 'no_match_name', 'percent_match'])
    for value1 in list1:
        for value2 in list2:
            match_ratio = fuzz.ratio(value1, value2)
            partial_match_ratio = fuzz.partial_ratio(value1, value2)
            insert(df, [value1, value2, match_ratio, partial_match_ratio])
    return df


<h2>Look into difflib from the standard library and possibly fuzzywuzzy to look for similar names</h2>

<a href='http://stackoverflow.com/questions/10383044/fuzzy-string-comparison'>Stack Overflow Answer</a>

<a href='https://en.wikipedia.org/wiki/Approximate_string_matching'>Aproximate String Algorithms</a>
<a href='http://www.nasdaq.com/screening/company-list.aspx'>Nasdaq Company List</a>
<p>Look at selenium library to do operations that need to be done in a browser</p>


<pre>
    # Add column example
    
    df = pd.DataFrame(columns=['col1', 'col2'])
    
    # This creates a dataframe with two columns
    # to add a third column simply use:
    
    df['col3'] = 'NaN'
    
    # This will create the third column and initialize it with the NaN value.
    # The new column can be initialized with any value you want
   

</pre>


In [368]:
#temp_filings = pd.DataFrame(filings_df)

#temp_csi = pd.DataFrame(csi_data)

#df = pd.DataFrame(columns=['filings_name', 'no_match_name', 'match_ratio', 'partial_match_ratio'])
#matching_phrases = check_for_matching_phrases(df, temp_filings['company_name'].values.tolist(), temp_csi['company_name'].values.tolist())

In [369]:
#df2 = df.sort_values(by=['match_ratio', 'partial_match_ratio'], ascending=False)
#df.sort_values(by=['percent_match'], ascending=False)

In [370]:
#df[df['partial_match_ratio'] >= 60]

In [386]:
nasdaq_companies = pd.read_csv("data/nasdaqcompanylist.csv")
amex_companies = pd.read_csv("data/amexcompanylist.csv")
nyse_companies = pd.read_csv("data/nysecompanylist.csv")

In [423]:
nasdaq_name_and_ticker = pd.DataFrame(nasdaq_companies, columns=['Symbol', 'Name'])
amex_name_and_ticker = pd.DataFrame(amex_companies, columns=['Symbol', 'Name'])
nyse_name_and_ticker = pd.DataFrame(nyse_companies, columns=['Symbol', 'Name'])
companies_and_symbols = pd.concat([nasdaq_name_and_ticker, amex_name_and_ticker, nyse_name_and_ticker])
companies_and_tickers = companies_and_symbols.rename(columns={'Symbol': 'ticker', "Name": "name"})

In [424]:
companies_and_tickers.head()

,ticker,name
0,PIH,"1347 Property Insurance Holdings, Inc."
1,FLWS,"1-800 FLOWERS.COM, Inc."
2,FCCY,1st Constitution Bancorp (NJ)
3,SRCE,1st Source Corporation
4,VNET,"21Vianet Group, Inc."


In [458]:
list_of_tickers_from_nasdaq = companies_and_tickers['ticker'].values.tolist()
list_of_tickers_in_cik_to_csi_table = pd.read_sql_query('select distinct symbol from cik_to_csi;',con=engine)['symbol'].values.tolist()


In [842]:
%%time
#ticker_list = make_list_of_tickers()

def tickers_to_cik(list_of_tickers):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile('.*CIK=(\d{10}).*')
    no_matches = []
    cik_dict = {}
    for ticker in list_of_tickers:
        results = CIK_RE.findall(get(URL.format(ticker)).text)
        if len(results):
            cik_dict[str(ticker).upper()] = str(results[0]).lstrip('0')
        else:
            no_matches.append(str(ticker).upper())
    return cik_dict, no_matches
ticker_cik_dict, no_matches = tickers_to_cik(list(set(ticker_list)))
print(len(ticker_cik_dict))

5
CPU times: user 624 ms, sys: 38.3 ms, total: 663 ms
Wall time: 19.3 s


In [860]:
no_matches

['VIN', 'PXLG', 'CDSUI', 'BCGR', 'TCHSF']

In [853]:
import json

In [856]:
'''
with open('data/ticker_to_cik.json', 'w') as f:
    f.write(json.dumps(str(ticker_cik_dict)))
    f.close()
with open('data/tickers_no_cik.txt', 'w') as f:    
    f.write(str(no_matches))
    f.close()
'''

"\nwith open('data/ticker_to_cik.json', 'w') as f:\n    f.write(json.dumps(str(ticker_cik_dict)))\n    f.close()\nwith open('data/tickers_no_cik.txt', 'w') as f:    \n    f.write(str(no_matches))\n    f.close()\n"

In [871]:
from urllib.parse import urlparse
from threading import Thread
import http.client as httplib 
import sys
from queue import Queue


def tickers_to_cik(list_of_tickers):
    concurrent = 200

    def doWork():
        while True:
            url = q.get()
            status, url = getStatus(url)
            doSomethingWithResult(status, url)
            q.task_done()

    def getStatus(ourl):
        try:
            url = urlparse(ourl)
            conn = httplib.HTTPConnection(url.netloc)   
            conn.request("HEAD", url.path)
            res = conn.getresponse()
            return res.status, ourl
        except:
            return "error", ourl

    def doSomethingWithResult(status, url):
        #print(status, url)
        pass

    q = Queue(concurrent * 2)
    for i in range(concurrent):
        t = Thread(target=doWork)
        t.daemon = True
        t.start()


    try:    
        URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
        CIK_RE = re.compile('.*CIK=(\d{10}).*')
        no_matches = []
        cik_dict = {}
        for ticker in list_of_tickers:
            results = CIK_RE.findall(get(URL.format(ticker)).text)
            q.put(results)
            if len(results):
                cik_dict[str(ticker).upper()] = str(results[0]).lstrip('0')
            else:
                no_matches.append(str(ticker).upper())
        q.join()


    except KeyboardInterrupt:
        sys.exit(1)
    return cik_dict, no_matches

In [882]:
ticker_list = make_list_of_tickers()
len(list(set(ticker_list)))

99340

In [ ]:
matches, non_matches = tickers_to_cik(ticker_list)